In [1]:
import gzip
import pandas as pd
import os


# Nombres de archivos
archivo_gz = 'clinvar.vcf.gz'
archivo_salida_tsv = 'archivo_descomprimido.tsv'

# Leer el archivo .vcf.gz
'''with gzip.open(archivo_gz, 'rt') as f:
    # Saltar metadatos al principio del archivo (líneas que empiezan con '##')
    lines = [line for line in f if not line.startswith('##')]
    
    # Crear un DataFrame a partir de las líneas restantes
    # La primera línea después de los metadatos es el encabezado (empieza con '#')
    df = pd.DataFrame([line.strip().split('\t') for line in lines])
    
    # La primera fila contiene los nombres de las columnas (pero con un '#' en la primera columna)
    df.columns = df.iloc[0].str.replace('^#', '', regex=True)  # Eliminar '#' del nombre de la primera columna
    df = df[1:]  # Descartar la fila del encabezado de los datos del DataFrame

# Mostrar el DataFrame y las columnas
print(df.head())  # Cambiado a head() para no imprimir todo el DataFrame
print(df.columns)'''


"with gzip.open(archivo_gz, 'rt') as f:\n    # Saltar metadatos al principio del archivo (líneas que empiezan con '##')\n    lines = [line for line in f if not line.startswith('##')]\n    \n    # Crear un DataFrame a partir de las líneas restantes\n    # La primera línea después de los metadatos es el encabezado (empieza con '#')\n    df = pd.DataFrame([line.strip().split('\t') for line in lines])\n    \n    # La primera fila contiene los nombres de las columnas (pero con un '#' en la primera columna)\n    df.columns = df.iloc[0].str.replace('^#', '', regex=True)  # Eliminar '#' del nombre de la primera columna\n    df = df[1:]  # Descartar la fila del encabezado de los datos del DataFrame\n\n# Mostrar el DataFrame y las columnas\nprint(df.head())  # Cambiado a head() para no imprimir todo el DataFrame\nprint(df.columns)"

In [2]:
'''columnas_seleccionadas = ['CHROM','POS', 'ID', 'REF', 'ALT', 'INFO']
df = df[columnas_seleccionadas]
columna_info = ['INFO']
df1 = df[columna_info]
# Verificar el resultado
print(df.head())
print(df1.head(1))'''


"columnas_seleccionadas = ['CHROM','POS', 'ID', 'REF', 'ALT', 'INFO']\ndf = df[columnas_seleccionadas]\ncolumna_info = ['INFO']\ndf1 = df[columna_info]\n# Verificar el resultado\nprint(df.head())\nprint(df1.head(1))"

In [3]:
'''# Dividir la columna 'INFO' en varias columnas basadas en los pares clave=valor
info_df = df['INFO'].str.split(';', expand=True)

# Crear un DataFrame vacío para las nuevas columnas
new_info_df = pd.DataFrame()

# Iterar sobre cada columna en info_df para separar las claves y valores
for column in info_df:
    # Dividir cada string en clave y valor
    split_data = info_df[column].str.split('=', expand=True)
    
    # Asignar la clave como nombre de columna y el valor como datos de la columna
    if len(split_data.columns) > 1:  # Verificar si realmente hay clave y valor
        new_info_df[split_data[0].iloc[0]] = split_data[1]

# Ahora new_info_df contiene las columnas deseadas según las claves en 'INFO'''

"# Dividir la columna 'INFO' en varias columnas basadas en los pares clave=valor\ninfo_df = df['INFO'].str.split(';', expand=True)\n\n# Crear un DataFrame vacío para las nuevas columnas\nnew_info_df = pd.DataFrame()\n\n# Iterar sobre cada columna en info_df para separar las claves y valores\nfor column in info_df:\n    # Dividir cada string en clave y valor\n    split_data = info_df[column].str.split('=', expand=True)\n    \n    # Asignar la clave como nombre de columna y el valor como datos de la columna\n    if len(split_data.columns) > 1:  # Verificar si realmente hay clave y valor\n        new_info_df[split_data[0].iloc[0]] = split_data[1]\n\n# Ahora new_info_df contiene las columnas deseadas según las claves en 'INFO"

In [4]:
'''# Concatenar las columnas originales con las nuevas columnas extraídas
final_df = pd.concat([df, new_info_df], axis=1)
# Guardar el DataFrame en un archivo .pickle
final_df.to_pickle("final_dataframe.pickle")'''


'# Concatenar las columnas originales con las nuevas columnas extraídas\nfinal_df = pd.concat([df, new_info_df], axis=1)\n# Guardar el DataFrame en un archivo .pickle\nfinal_df.to_pickle("final_dataframe.pickle")'

In [5]:
loaded_df = pd.read_pickle("final_dataframe.pickle")
# Imprimir las primeras filas para revisar el DataFrame final
#renombra la columna
loaded_df.rename(columns={'ID': 'clinvar_id'}, inplace=True)


In [6]:
# Intentar dividir la columna 'GENEINFO', añadiendo un valor predeterminado para los casos sin delimitador ':'
temp_df = loaded_df['GENEINFO'].str.split(':', expand=True)
temp_df[1] = temp_df[1].fillna("None")  # Añadir 'Desconocido' si no hay segundo elemento

In [7]:
loaded_df['gene_symbol'] = temp_df[0]
loaded_df['gene_id'] = temp_df[1]

In [8]:
registros_sin_colon = loaded_df[~loaded_df['GENEINFO'].str.contains(':')]

print("Registros donde 'GENEINFO' no contiene el delimitador ':':")
#print(registros_sin_colon)

Registros donde 'GENEINFO' no contiene el delimitador ':':


In [9]:
print(registros_sin_colon.head(10))

   CHROM     POS clinvar_id REF ALT ALLELEID CLNDISDB    CLNDN  \
30     1  930165    1164676   G   A  0.00102  0.00280  1153701   
32     1  930187    1144630   C   T  0.00016  0.00005  1131738   
36     1  930195    1354167   C   T  0.00008  0.00004  1341285   
38     1  930200    1043045   G   A  0.00039  0.00038  1023510   
41     1  930204    1170208   G   A  0.01083  0.00595  0.01098   
54     1  930245    1131472   G   A  0.00069  0.00066  0.00140   
56     1  930248     789256   G   A  0.00347  0.00622  0.00280   
69     1  930282    1140293   G   A  0.00146  0.00064  0.00080   
70     1  930284    1435698   C   T  0.00008  0.00013  0.00020   
71     1  930285    1165489   G   A  0.00062  0.00094  1153703   

                                         CLNHGVS  \
30                             .|MedGen:C3661900   
32                               MedGen:C3661900   
36                               MedGen:C3661900   
38  MedGen:CN517202|MeSH:D030342,MedGen:C0950123   
41           

In [10]:
print(loaded_df.head(1))

  CHROM    POS clinvar_id REF ALT ALLELEID                      CLNDISDB  \
1     1  69134    2205837   A   G  2193183  MeSH:D030342,MedGen:C0950123   

                     CLNDN                  CLNHGVS  \
1  Inborn_genetic_diseases  NC_000001.11:g.69134A>G   

                            CLNREVSTAT         CLNSIG  \
1  criteria_provided,_single_submitter  Likely_benign   

                       CLNVC     CLNVCSO     GENEINFO  \
1  single_nucleotide_variant  SO:0001483  OR4F5:79501   

                            MC ORIGIN gene_symbol gene_id  
1  SO:0001583|missense_variant      1       OR4F5   79501  


In [11]:
specific_row = loaded_df.iloc[131]
print(specific_row)

CHROM                                                          1
POS                                                       935788
clinvar_id                                               1297123
REF                                                            C
ALT                                                            T
ALLELEID                                                 1286912
CLNDISDB       Human_Phenotype_Ontology:HP:0000547,MONDO:MOND...
CLNDN                                       Retinitis_pigmentosa
CLNHGVS                                 NC_000001.11:g.935788C>T
CLNREVSTAT                   criteria_provided,_single_submitter
CLNSIG                                    Uncertain_significance
CLNVC                                  single_nucleotide_variant
CLNVCSO                                               SO:0001483
GENEINFO                                           SAMD11:148398
MC                                   SO:0001583|missense_variant
ORIGIN                   

In [12]:
# Añadir nuevas columnas que voy a necesitar
loaded_df['DB'] = 'ClinVar' # Columna base de datos

In [13]:
specific_row_2 = loaded_df.iloc[1224564]
print(specific_row_2)

CHROM                                                          9
POS                                                      2115857
clinvar_id                                               1309084
REF                                                            G
ALT                                                            A
ALLELEID                                                 1301112
CLNDISDB            MeSH:D030342,MedGen:C0950123|MedGen:C3661900
CLNDN                       Inborn_genetic_diseases|not_provided
CLNHGVS                                NC_000009.12:g.2115857G>A
CLNREVSTAT     criteria_provided,_multiple_submitters,_no_con...
CLNSIG                                             Likely_benign
CLNVC                                  single_nucleotide_variant
CLNVCSO                                               SO:0001483
GENEINFO                                            SMARCA2:6595
MC                                 SO:0001819|synonymous_variant
ORIGIN                   

In [14]:
specific_row_3 = registros_sin_colon.iloc[100145]
print(specific_row_3)

CHROM                                                          5
POS                                                    148136985
clinvar_id                                                513233
REF                                                            C
ALT                                                            T
ALLELEID                                                 0.00008
CLNDISDB                                                 0.00027
CLNDN                                                     500709
CLNHGVS        MedGen:C3661900|MONDO:MONDO:0043106,MedGen:C02...
CLNREVSTAT     not_provided|Ichthyosis_linearis_circumflexa|n...
CLNSIG                               NC_000005.10:g.148136985C>T
CLNVC          criteria_provided,_multiple_submitters,_no_con...
CLNVCSO                                            Likely_benign
GENEINFO                               single_nucleotide_variant
MC                                                    SO:0001483
ORIGIN                   

In [16]:
# Define la ruta y el nombre del archivo TSV de salida
archivo_salida_tsv = 'salida_ClinVar.tsv'
carpeta_salida = 'Salidas'
archivo_salida_tsv = os.path.join(carpeta_salida, archivo_salida_tsv)
# Guardar el DataFrame en un archivo TSV
loaded_df.to_csv(archivo_salida_tsv, sep='\t', index=False)

print(f"DataFrame guardado con éxito en '{carpeta_salida}'")

DataFrame guardado con éxito en 'Salidas'
